In [9]:
import psycopg2

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database='final_project',
    user='postgres',
    password='PASSWORD',
    port='5432'
)

curr = conn.cursor()

In [16]:
curr.execute(
    '''
    CREATE TABLE IF NOT EXISTS ticker_data (
        date DATE,
        ticker VARCHAR(50),
        open NUMERIC(10, 3),
        high NUMERIC(10, 3),
        low NUMERIC(10, 3),
        close NUMERIC(10, 3),
        volume NUMERIC(15,2),
        PRIMARY KEY (date, ticker)
    )
    '''
)

In [17]:
conn.commit()

In [18]:
curr.execute("SELECT * FROM ticker_data;")

In [19]:
with open('all_tickers_data.csv', 'r') as f:
    next(f)  # Skip the header row

    curr.copy_from(f, 'ticker_data', sep=',', columns=('date', 'ticker', 'open', 'high', 'low', 'close', 'volume'))

conn.commit()

In [20]:
curr.close()

In [25]:
curr = conn.cursor()
curr.execute('''
            CREATE TABLE IF NOT EXISTS tickers_info (
            ticker VARCHAR(50) PRIMARY KEY,
            currency VARCHAR(10),
            exchange VARCHAR(50),
            exchangeTimezoneName VARCHAR(50),
            exchangeTimezoneShortName VARCHAR(50),
            fullExchangeName VARCHAR(50),
            gmtOffSetMilliseconds INTEGER,
            longName VARCHAR(100),
            market VARCHAR(50),
            marketState VARCHAR(50),
            quoteType VARCHAR(50),
            region VARCHAR(50),
            shortName VARCHAR(100),
            symbol VARCHAR(50),
            typeDisp VARCHAR(50)
        )
    ''')

conn.commit()

In [32]:
with open('filtered_tickers_data.csv', 'r') as f:
    next(f)  # Skip the header row

    curr.copy_from(f, 'tickers_info', sep=',', columns=('ticker', 'currency', 'exchange', 'exchangetimezonename',
                                                        'exchangetimezoneshortname', 'fullexchangename',
                                                        'gmtoffsetmilliseconds', 'longname', 'market',
                                                        'marketstate', 'quotetype', 'region', 'shortname', 'symbol',
                                                        'typedisp'))
                                                        
    
conn.commit()

In [34]:
curr.close()


In [38]:
curr = conn.cursor()
curr.execute('''
    CREATE TABLE IF NOT EXISTS fund_details (
        fund_id INTEGER PRIMARY KEY,
        fund_house VARCHAR(100),
        scheme_type VARCHAR(100),
        scheme_category VARCHAR(100),
        scheme_code INTEGER,
        scheme_name VARCHAR(200),
        isin_growth VARCHAR(50)
             )
        ''')
             
conn.commit()

with open('dim_fund.csv', 'r') as f:
    next(f)
    curr.copy_from(f, 'fund_details', sep=',', columns=('fund_id', 'fund_house', 'scheme_type', 'scheme_category',
                                                        'scheme_code', 'scheme_name', 'isin_growth'))

conn.commit()

curr.close()


In [44]:
curr = conn.cursor()
curr.execute('''
    CREATE TABLE IF NOT EXISTS fund_data (
        fund_id INTEGER,
        date DATE,
        nav NUMERIC(10, 3)
             )
             ''')

with open('fact_fund_prices.csv', 'r') as f:
    next(f)
    curr.copy_from(f, 'fund_data', sep=',', columns=('fund_id', 'date', 'nav'))

conn.commit()

curr.close()

In [ ]:
curr = conn.cursor()
curr.execute('''
    CREATE TABLE IF NOT EXISTS fund_holdings (
        stock_name VARCHAR(100),
        isin VARCHAR(50),
        industry VARCHAR(100),
        quantity NUMERIC(15, 2),
        market_value_in_lacs NUMERIC(15, 2),
        scheme_code INTEGER,
        scheme_name VARCHAR(200)
    )
''')

# Reading using copy_expert to handle CSV with quotes
with open('fact_fund_holdings.csv', 'r', encoding='utf-8') as f:
    # Use copy_expert so Postgres respects CSV quotes
    curr.copy_expert("""
        COPY fund_holdings (stock_name, isin, industry, quantity,
                            market_value_in_lacs, scheme_code, scheme_name)
        FROM STDIN
        WITH (FORMAT csv, HEADER true, QUOTE '"')
    """, f)

conn.commit()


In [73]:
curr.close()
conn.close()
